In [1]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import numpy as np

import pickle
from scipy.integrate import simps
from scipy.interpolate import RectBivariateSpline
from classy import Class
import sys
sys.path.append("../scripts/") 
from postBornCrossSpectra import PostBorn_Bispec
import Cosmology as C
import kernels
plt.style.use('classic')

In [2]:
from lab import *

In [3]:
plotpath = '../output/plots/'

In [4]:
zmin = 1e-5
zmax = 1090.

kmin = 1e-4
kmax = 50

cosmo_dict.pop('z_cmb')
cosmo = [{'name':"EmasCosmo"},cosmo_dict]

print(cosmo_dict)

a     = np.linspace(1./(1.+zmin),1./(1.+zmax),500)  
z     = 1./a-1.

data  = C.CosmoData(cosmo[1],z)

L     = np.logspace(1,4,400)

# sampling for integrations
phi = np.linspace(0.,2.*np.pi,1000,endpoint=True)
L1  = np.logspace(-1,5,400)

LSST_bins  = [0,1,2,3,4,'all']

{'A_s': 2.10732e-09, 'h': 0.677, 'k_pivot': 0.05, 'n_s': 0.96824, 'omega_b': 0.02247, 'omega_cdm': 0.11923}
computing distances, derived parameters...
{'Neff': 3.046}
{'h': 0.677}
{'m_ncdm_tot': 0.0}
{'omega_m': 0.6745515980724002}


In [5]:
# define redshift bins (LSST-like)
zmins= [0,0.5,1,2,3,0]
zmaxs= [0.5,1.,2.,3.,4.,'z$_{\mathrm{CMB}}$']
ngal = [14.3,23.9,21.6,3.7,1.5,65.7]

colors=['navy','maroon', 'darkgreen', 'mediumaquamarine','indianred','darkorange']

In [6]:
#compute correction following Krause&Hirata
#always use trapz rule for accurate results!
#redefinition with l'-> l'-l
def get_regularized_sum_of_terms(L,L1,phi,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            L_= LL*np.sqrt(1.+(LL1/LL)**2+2.*LL1/LL*nu)
            integrand =LL*LL1**3*(LL1*nu+LL)*(LL1+LL*nu)**2*(M(L_,LL1,grid=False)/L_**2/LL1**4-M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res


#numerical angular integration
def get_second_sum_of_terms(L,L1,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            integrand = LL1*(LL*LL1**2*(-LL1*nu+LL)*(-LL1+LL*nu)**2-LL**2*(LL*LL1*nu)**2)*(M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

In [7]:
first_kernel  = kernels.CMB_lens(data.chi_cmb,data)
simple_kernel = kernels.CMB_lens(None,data)

Mstarspls = []
PBs =[]
Cls= []
for LSST_bin in LSST_bins:
    second_kernel = kernels.gal_clus(kernels.dNdz_LSST,kernels.simple_bias,data,LSST_bin)
    PB  = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=kmin,k_max=100, lmax=30000, acc=2)
    PBs+=[PB]
    Mstarspls+=[PB.Mstarsp]
    Cls+=[PB.CL_born]
ls = PB.ls


using z-bin [0, 0.5, 0.03] norm 14.32276056640902
Initializing CLASS with halofit...
{'A_s': 2.10732e-09, 'h': 0.677, 'k_pivot': 0.05, 'n_s': 0.96824, 'omega_b': 0.02247, 'omega_cdm': 0.11923, 'output': 'mPk', 'P_k_max_1/Mpc': 101, 'z_max_pk': 1088.6464513003662, 'k_min_tau0': 0.002, 'k_max_tau0_over_l_max': 5.0, 'k_step_sub': 0.015, 'k_step_super': 0.0001, 'k_step_super_reduction': 0.1, 'k_per_decade_for_pk': 20, 'perturb_sampling_stepsize': 0.01, 'tol_perturb_integration': 1e-06, 'halofit_k_per_decade': 3000.0, 'non linear': 'halofit'}
sigma8  0.822427245205793
zmax 1088.6464513003662
using z-bin [0.5, 0.5, 0.03] norm 23.943209238913052
Initializing CLASS with halofit...
{'A_s': 2.10732e-09, 'h': 0.677, 'k_pivot': 0.05, 'n_s': 0.96824, 'omega_b': 0.02247, 'omega_cdm': 0.11923, 'output': 'mPk', 'P_k_max_1/Mpc': 101, 'z_max_pk': 1088.6464513003662, 'k_min_tau0': 0.002, 'k_max_tau0_over_l_max': 5.0, 'k_step_sub': 0.015, 'k_step_super': 0.0001, 'k_step_super_reduction': 0.1, 'k_per_decad

In [8]:
corrs=[]
for ii in range(len(LSST_bins)):
    print(ii),
    corrs+=[get_regularized_sum_of_terms(L,L1,phi,Mstarspls[ii],method='trapz')+get_second_sum_of_terms(L,L1,Mstarspls[ii],method='trapz')]

0
1
2
3
4
5


In [9]:
corrs = np.asarray(corrs)

In [10]:
corrs

array([[ 3.46265811e-13,  3.42074111e-13,  3.37396031e-13, ...,
        -1.13756037e-14, -1.07529190e-14, -1.01572956e-14],
       [ 1.23640950e-12,  1.25594445e-12,  1.27528572e-12, ...,
        -1.86378438e-13, -1.80217976e-13, -1.74156689e-13],
       [ 1.81325605e-12,  1.85583755e-12,  1.89901601e-12, ...,
        -5.28531598e-13, -5.20855694e-13, -5.13082268e-13],
       [ 1.99743162e-12,  2.05122685e-12,  2.10616487e-12, ...,
        -4.65225474e-13, -4.61876622e-13, -4.58397171e-13],
       [ 1.94239857e-12,  1.99732048e-12,  2.05354627e-12, ...,
        -2.91619795e-13, -2.85940518e-13, -2.81110204e-13],
       [ 1.29351746e-12,  1.31840766e-12,  1.34342168e-12, ...,
        -2.78320816e-13, -2.73028055e-13, -2.67766258e-13]])

In [11]:
np.save('../output/LimberPostBornResultsLSST',(L,corrs))


In [12]:
L,corrs = np.load('../output/LimberPostBornResultsLSST.npy')